In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import spacy
import nltk
import requests
import certifi
from nltk.corpus import brown
nltk.download('brown')

nlp = spacy.load("en_core_web_sm")
sw_spacy = nlp.Defaults.stop_words

from nltk.stem.porter import *
from collections import defaultdict

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Harsha\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [2]:
vocab = set(brown.words())

stemmer = PorterStemmer()
d = defaultdict(set)
for v in vocab:
    d[stemmer.stem(v)].add(v)  

In [4]:
possible = d[stemmer.stem('letting')]
for p in possible:
    print(p)

letting
Letting
Let
lets
let


In [3]:
def check_noun(word):
    doc = nlp(word)

    if(doc[0].tag_ == 'NNP'):
        return True
    else:
        return False

check_noun("chakri")

True

In [6]:
test = ['free', 'will', 'changing', 'destiny', 'choosing', 'love', 'choice', 'soul', 'conditioning', 'acceptance']

if 'soul' in test:
    print('works')

works


In [4]:
def remove_stopwords(text):

    # words = [word for word in text.split() if word.lower() not in sw_spacy]
    # new_text = " ".join(words)
    # print(new_text)
    # print("Old length: ", len(text))
    # print("New length: ", len(new_text))

    res = []
    for word in text:
        if word not in sw_spacy:
            res.append(word)
    return res


In [9]:


page = urlopen(
    "https://ezinearticles.com/?Live-In-the-Present-to-Create-the-Life-You-Desire&id=6504152",
    context=ssl.create_default_context(cafile=certifi.where()))


soup = BeautifulSoup(page, 'html.parser')
title = soup.title.string.lower().split()
key = soup.select("meta[name='keywords']")[0]['content'].lower().split(",")

print(title)
keywords = [] 
for k in key:
    
    for word in k.split():
        if word not in keywords:
            keywords.append(word)

result=[]

print(keywords)

for word in title:
    if check_noun(word):
        if word not in result:
            result.append(word)

for word in keywords:
    possible = d[stemmer.stem(word)]
    for p in possible:
        if p in title:
            if p not in result:
                result.append(p)
        
result= remove_stopwords(result)
print(result)

['live', 'in', 'the', 'present', 'to', 'create', 'the', 'life', 'you', 'desire']
['the', 'present', 'living', 'in', 'now', 'gift', 'of', 'moment', 'unknown', 'future', 'painful', 'past']
['present', 'live']


In [12]:
url="https://ezinearticles.com/?Live-In-the-Present-to-Create-the-Life-You-Desire&id=6504152"
requests.get(url,verify=True)

<Response [403]>

In [35]:
page = urlopen(
    "https://ezinearticles.com/?Live-In-the-Present-to-Create-the-Life-You-Desire&id=6504152",
    context=ssl.create_default_context(cafile=certifi.where()))


soup = BeautifulSoup(page, 'html.parser')
name = 'ezinearticles'
found = []
for a in soup.select('a[href]'):
    b = a['href']
    if 'https://' + name + '.com' in b and b not in found:
        found.append(b)
        break

print(found)




[]


In [5]:
def full_url(partial_url):
    return "https://ezinearticles.com"+partial_url

In [64]:
mydivs = soup.find_all("ul", {"class": "ea-similar-articles ea-article-list"})[0]
type(mydivs)
#mydivs
test1 = mydivs.findChildren('a')
for i in range(0,len(test1)):
    tag = test[i]
    print(full_url(tag.get_attribute_list('href')[0]))


https://ezinearticles.com/?The-Arsenal-Team-of-the-Decade-(1997-2007)&id=606410
https://ezinearticles.com/?THE-MOST-HIGH-GOD-~-Fire-Baptisms-and-the-Anointed-Ones-(The-Cannabis-Revelation)&id=10467079
https://ezinearticles.com/?Nirut-and-the-She-Ocean-(and-How-the-Nirut-Series-was-Written)&id=644444
https://ezinearticles.com/?The-Value-of-the-Gift-of-Life&id=6507101
https://ezinearticles.com/?Living-in-the-Present-Moment&id=1251181
https://ezinearticles.com/?Tazaungmon-And-The-Tazaungdaing-Light-Festival&id=9075281
https://ezinearticles.com/?The-Difference-Between-Being-Smart,-Educated,-and-Intelligent&id=2181806
https://ezinearticles.com/?Finding-Your-Next-Step-in-the-Midst-of-Fear,-Pain,-Or-Chaos&id=1640380
https://ezinearticles.com/?The-Aphrodite-Project---A-Discovery-of-Life-in-Cancer-Survivors&id=3478385
https://ezinearticles.com/?The-Essentials-of-a-Good-Life&id=1874482


In [62]:
type(test[0].get_attribute_list('href')[0])

str

In [47]:

def spider(found_titles, url, found):
    try:
        page = urlopen(url,context=ssl.create_default_context(cafile=certifi.where()))
        soup = BeautifulSoup(page, 'html.parser')

        og_title = soup.title.string.lower()
        title = soup.title.string.lower().split()
        key = soup.select("meta[name='keywords']")[0]['content'].lower().split(",")
        keywords = [] 
        for k in key:
            for word in k.split():
                if word not in keywords:
                    keywords.append(word)

        result=[]

        print(keywords)

        for word in title:
            if check_noun(word):
                if word not in result:
                    result.append(word)

        for word in keywords:
            possible = d[stemmer.stem(word)]
            for p in possible:
                if p in title:
                    if p not in result:
                        result.append(p)
        result= remove_stopwords(result)
        
        if len(result) > 0 and og_title not in found_titles:
            found_titles.append(og_title)
            print(og_title)
            print(result)

            f = open('test.txt', 'a')
            f.write(
                og_title + "\t" + ' '.join(
                    k.replace(' ', '_') for k in result
                ) + "\n"
            )
            f.close()
        
        mydivs = soup.find_all("ul", {"class": "ea-similar-articles ea-article-list"})[0]
        #print(mydivs)
        
        if mydivs:
            test1 = mydivs.findChildren('a')
            print("here: ",test1)
            if test1:
                for i in range(0,len(test1)):
                    tag = test1[i]
                    
                    url_here = full_url(tag.get_attribute_list('href')[0])
                    print("url here: ",url_here)
                    if url_here not in found:
                        found.append(url_here)
                        spider(found_titles,url_here,found)
        
    except:
        pass

In [50]:
start_url = [
    "https://ezinearticles.com/?Uses-of-Silicon-Carbide-Ceramics-in-Different-Industries&id=10550034",
    "https://ezinearticles.com/?Some-Dangers-From-Pandemic-Fatigue&id=10554149",
    "https://ezinearticles.com/?What-Might-Be-Next-In-The-Economy?&id=10554161",
    "https://ezinearticles.com/?Appointments---Manage-Your-Time-Better-At-Home-to-Be-Effective&id=10553717",
    "https://ezinearticles.com/?Appointments---The-Importance-of-Making-Appointments-for-Entrepreneurs&id=1055409"
    ]

found = [start_url[0]]
found_titles = []
for starter in start_url:
    spider(found_titles,starter,found)

['silicon', 'carbide', 'nozzle', 'ceramics', 'ceramic']
uses of silicon carbide ceramics in different industries
['carbide', 'silicon', 'ceramics']
here:  [<a href="/?All-About-Silicon-Carbide-Ceramic&amp;id=9875605">
<span>All About Silicon Carbide Ceramic</span>
</a>, <a href="/?Low-Cost-Synthesis-of-Silicon-Carbide-and-Silicon-Nitride&amp;id=7087791">
<span>Low Cost Synthesis of Silicon Carbide and Silicon Nitride</span>
</a>, <a href="/?Reaction-Bonded-Silicon-Carbide:-Advantages,-Application-and-Products&amp;id=10543934">
<span>Reaction Bonded Silicon Carbide: Advantages, Application and Products</span>
</a>, <a href="/?Everything-About-Sintered-Silicon-Carbide&amp;id=9666615">
<span>Everything About Sintered Silicon Carbide</span>
</a>, <a href="/?Know-the-Several-Applications-of-Silicon-Carbide&amp;id=9786591">
<span>Know the Several Applications of Silicon Carbide</span>
</a>, <a href="/?Ceramic-Tile-Types---Your-Complete-Electrical-Solution&amp;id=9068929">
<span>Ceramic Tile 